In [128]:
import matplotlib.pyplot as plt
from pydub import AudioSegment
import numpy as np
import cv2
import glob
import os
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
from pathlib import Path
import math


In [129]:
def calculate_decibel(audio):
    samples = np.array(audio.get_array_of_samples())
    rms = np.sqrt(np.mean(np.square(samples)))

    if rms == 0 or np.isnan(rms) or np.isinf(rms):
        return float('-inf')

    reference = 1.0
    decibel = 20 * np.log10(rms / reference)

    return decibel

In [130]:
def get_decibel_levels(audio_path):
    audio = AudioSegment.from_file(audio_path)
    decibel_levels = [calculate_decibel(audio[i:i+1]) for i in range(0, len(audio), 1)]
    return decibel_levels

In [131]:
def process_decibel_levels(decibel_levels):
    for i in range(len(decibel_levels)):
        if math.isinf(decibel_levels[i]):
            decibel_levels[i] = 0
        else:
            decibel_levels[i] = int(decibel_levels[i])

    avg_decibel = sum(decibel_levels) / len(decibel_levels)
    max_decibel = max(decibel_levels)
    # second_high_decibel = decibel_levels.sort()[-2]

    return avg_decibel, max_decibel, decibel_levels

In [132]:
# def assign_images(decibel_levels, avg_decibel, max_decibel):
#     images = []
#     for i in range(len(decibel_levels)):
#         if decibel_levels[i] == 0:
#             images.append('./gwack/Line.png')
#         elif 0 <= decibel_levels[i] < avg_decibel / 2:
#             images.append('./gwack/Teeth.png')
#         elif avg_decibel / 2 <= decibel_levels[i] < avg_decibel:
#             images.append("./gwack/big.png")
#         elif decibel_levels[i] == avg_decibel:
#             images.append('./gwack/circle.png')
#         elif avg_decibel < decibel_levels[i] <= max_decibel:
#             images.append('./gwack/Biggest.png')
#     return images

In [133]:
# def assign_images(decibel_levels, avg_decibel, max_decibel):
#     images = []
#     for level in decibel_levels:
#         if level == 0:
#             images.append('./gwack/Line.png')
#         elif 0 <= level < avg_decibel / 4:
#             images.append('./gwack/Teeth.png')
#         elif avg_decibel / 4 <= level < avg_decibel * 3 / 4:
#             images.append("./gwack/big.png")
#         elif level >= avg_decibel * 3 / 4:
#             images.append('./gwack/Biggest.png')
#     return images

In [134]:
# def assign_images(decibel_levels, avg_decibel, max_decibel, window_size=40):
#     smoothed_levels = np.convolve(decibel_levels, np.ones(window_size)/window_size, mode='same')
    
#     images = []
#     for level in smoothed_levels:
#         if level == 0:
#             images.append('./gwack/Line.png')
#         elif 0 <= level < avg_decibel / 4:
#             images.append('./gwack/Teeth.png')
#         elif avg_decibel / 4 <= level < avg_decibel * 3 / 4:
#             images.append("./gwack/big.png")
#         elif level >= avg_decibel * 3 / 4:
#             images.append('./gwack/Line.png')
#     return images


In [135]:
def assign_images(decibel_levels, avg_decibel, max_decibel, window_size=40):
    smoothed_levels = np.convolve(decibel_levels, np.ones(window_size)/window_size, mode='same')
    
    images = []
    for level in smoothed_levels:
        if level == 0:
            images.append('./gwack/line.png')
        elif 0 <= level < avg_decibel / 4:
            images.append('./gwack/Teeth_ee.png')  
        elif avg_decibel / 4 <= level < avg_decibel * 3 / 4:
            images.append('./gwack/Teeth.png')
        elif level >= avg_decibel * 3 / 4:
            images.append('./gwack/line.png')
    return images


In [136]:
def img_array(arr):
    img_array = []
    for i in arr:
        img = cv2.imread(i)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    return img_array

In [137]:
def create_video(images, output_path, codec='DIVX', fps=1000):
# def create_video(images, output_path, codec='DIVX', fps=25):
    animation = img_array(images)
    frame_size = (animation[0].shape[1], animation[0].shape[0])

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*codec), fps, frameSize=frame_size)

    for i in range(len(animation)):
        out.write(animation[i])

    out.release()

In [138]:
# def create_final_video(video_path, audio_path, output_path, title="sample"):
#     video_clip = VideoFileClip(video_path, fps=25)
def create_final_video(video_path, audio_path, output_path, title="sample"):
    video_clip = VideoFileClip(video_path)
    audio_clip = AudioFileClip(audio_path)
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_path + ".mp4")


In [139]:
def main(path):
    audio_path = path
    decibel_levels = get_decibel_levels(audio_path)

    avg_decibel, max_decibel, processed_decibel_levels = process_decibel_levels(decibel_levels)

    print("Processed Decibel Levels:", processed_decibel_levels)
    print("Average Decibel Level:", avg_decibel)
    print("Max Decibel Level:", max_decibel)

    assigned_images = assign_images(processed_decibel_levels, avg_decibel, max_decibel)
    print("Assigned Images:", assigned_images)

    video_output_path_avi = 'project.avi'
    video_output_path_mp4 = 'project.mp4'

    create_video(assigned_images, video_output_path_avi, codec='DIVX', fps=1000)
    create_video(assigned_images, video_output_path_mp4, codec='mp4v', fps=1000)

    create_final_video(video_output_path_avi, audio_path, 'final_output_avi', title="sample")
    create_final_video(video_output_path_mp4, audio_path, 'final_output_mp4', title="sample")

In [140]:
if __name__ == "__main__":
    main("./sound/Ashta 4.m4a")
    print("Finished!")

Processed Decibel Levels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -8, -3, -2, 1, 1, 0, 2, -2, -3, -1, 1, 14, 20, 23, 26, 28, 17, 21, 25, 17, 30, 28, 30, 34, 30, 31, 34, 31, 25, 24, 34, 33, 25, 25, 21, 34, 33, 25, 30, 33, 31, 34, 25, 26, 36, 32, 36, 32, 31, 36, 36, 35, 37, 39, 39, 33, 28, 32, 27, 26, 31, 36, 41, 37, 37, 40, 40, 40, 39, 33, 40, 39, 30, 35, 38, 31, 34, 32, 30, 37, 32, 30, 37, 36, 42, 40, 35, 30, 32, 37, 35, 34, 40, 37, 40, 38, 31, 33, 40, 32, 0, 40, 34, 32, 38, 40, 40, 34, 38, 36, 33, 38, 40, 0, 31, 35, 0, 35, 38, 0, 0, 0, 34, 38, 23, 0, 36, 33, 24, 36, 26, 34, 33, 36, 0, 39, 0, 39, 34, 0, 34, 0, 0, 31, 33, 32, 0, 38, 32, 38, 0, 40, 33, 29, 38, 0, 0, 37, 0, 0, 36, 0, 31, 30, 36, 34, 32, 35, 0, 35, 35, 40, 33, 36, 0, 31, 33, 37, 0, 0, 30, 0, 32, 36, 36, 39, 38, 37, 0, 0, 31, 22, 34, 36, 37, 0, 40, 40, 40, 37, 38, 37, 35, 0, 0, 31, 36, 40, 26, 0, 35, 20, 37, 36, 24, 0, 0, 31, 30, 35

libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on gr

Moviepy - Building video final_output_avi.mp4.
MoviePy - Writing audio in final_output_aviTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_output_avi.mp4



Moviepy - Done !
Moviepy - video ready final_output_avi.mp4
Moviepy - Building video final_output_mp4.mp4.
MoviePy - Writing audio in final_output_mp4TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_output_mp4.mp4



Moviepy - Done !
Moviepy - video ready final_output_mp4.mp4
Finished!
